In [ ]:
from glob import glob
import sys

sys.path.append('../')
from source_names_dict import source_names_dict, source_names_readable, source_distances_mpc, source_distances_mpc, get_simbad_name_glob
from get_closest_srcreg import read_region_file

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.gridspec as gridspec

from astropy.coordinates import SkyCoord
from astropy.wcs.utils import skycoord_to_pixel
from astropy.wcs import WCS
from astropy.io import fits
from astroquery.simbad import Simbad
import astropy.units as u
from get_closest_srcreg import get_src_region_dict
from get_all_region_files import get_all_regions
from astropy.nddata import Cutout2D
from astropy.nddata.utils import NoOverlapError
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from astropy.wcs import FITSFixedWarning
import warnings
warnings.filterwarnings('ignore', category=FITSFixedWarning, append=True)

In [ ]:
src_region_dict, df_closest = get_src_region_dict(return_df=True)

In [ ]:
df_closest

In [ ]:
def plot_finding_chart(simbad_name, local_name, xrt_img_path, uvot_img_path, df_closest):
    
    # Get UVOT source region
    try:
        sc_uvot_reg = df_closest[df_closest['simbad_name'] == simbad_name]['local_sc'].iloc[0]
        uvot_reg = True
    except Exception as e:
        uvot_reg = False
        print(f'Could not get uvot source region for {simbad_name}')
        print(e)
    
    # Get readable name
    readable_name = source_names_readable[simbad_name]
    source_distance = source_distances_mpc[simbad_name]
    
    # Get filter
    uvot_filter = uvot_img_path.split('/')[-1][:-4].split('_')[-1].upper()
    
    # Get source position from simbad
    s = Simbad()
    source = s.query_object(simbad_name)
    sc = SkyCoord(source['RA'], source['DEC'], unit=(u.hourangle, u.deg))

    hdul = fits.open(xrt_img_path)
    hdul_uv = fits.open(uvot_img_path)

    xrt_threshold = 10
    cutout_size   = 50

    img_data = hdul[0].data
    # mask the XRT array to get rid of background
    sources = np.ma.masked_where(img_data < xrt_threshold, img_data)


    # Create UV cutout region around source
    xpos_xrt, ypos_xrt = skycoord_to_pixel(sc, WCS(hdul[0].header))
    xpos, ypos = skycoord_to_pixel(sc, WCS(hdul_uv[1].header))
    pos  = (xpos[0], ypos[0])
    cutout = Cutout2D(hdul_uv[1].data, pos, cutout_size, wcs=WCS(hdul_uv[1].header)) # 50 pixel square

    title = f'{simbad_name} | {local_name} | {readable_name}\nRA={str(sc.ra[0])} DEC={str(sc.dec[0])}\nUVOT_FILTER={uvot_filter} | d = {source_distance} Mpc'
    
    # Plotting
    fig = plt.figure(figsize=(10, 10))
    plt.suptitle(title)
    ax = plt.subplot(projection=WCS(hdul[0].header))
    ax.imshow(hdul_uv[1].data, norm=LogNorm(10), cmap='hot', transform=ax.get_transform(WCS(hdul_uv[1].header)), origin='lower', interpolation='none')
    ax.imshow(sources, norm=LogNorm(10), cmap='winter', interpolation='none', origin='lower')

    cutout.plot_on_original(color='white', transform=ax.get_transform(WCS(hdul_uv[1].header)))
    ax.axvline(xpos_xrt, lw=1.0, color='cyan')
    ax.axhline(ypos_xrt, lw=1.0, color='cyan')
    ax.annotate(xy=(xpos_xrt+15,ypos_xrt+5), text=readable_name, color='cyan')

    # Zoom in a bit
    extent = 200 # Extent in pixels
    ax.set_xlim(xpos_xrt-extent, xpos_xrt+extent)
    ax.set_ylim(ypos_xrt-extent, ypos_xrt+extent)
    
    ax.set_xlabel('RA (h:m:s)')
    ax.set_ylabel('DEC (d:m:s)')

    # Plot the inset
    left, bottom, width, height = [0.6, 0.6, 0.25, 0.25]
    ax2 = fig.add_axes([left, bottom, width, height], projection=cutout.wcs)
    ax2.imshow(cutout.data, origin='lower', aspect='equal', norm=LogNorm(), interpolation='none', cmap='hot')

    xpos, ypos = skycoord_to_pixel(sc, cutout.wcs)
    
    
    # Plot crosshair
    ax2.axvline(xpos, color='cyan', lw=1.0)
    ax2.axhline(ypos, color='cyan', lw=1.0, label='SIMBAD')
    
    if uvot_reg:
        xpos_uvot_src, ypos_uvot_src = skycoord_to_pixel(sc_uvot_reg, cutout.wcs)
        ax2.axvline(xpos_uvot_src, color='magenta', lw=1.0)
        ax2.axhline(ypos_uvot_src, color='magenta', lw=1.0, label='UVOT')
    
    ax2.set_xlabel(' ')
    ax2.set_ylabel(' ')
    ax2.tick_params(axis='both', colors='white')
    ax2.legend(loc='upper left')
    
    #plt.savefig(f'../figures/finding_plots/{simbad_name}_{uvot_img_path.split("/")[-1][:-4]}.png')
    #plt.savefig(f'../figures/finding_plots/{simbad_name}_{uvot_img_path.split("/")[-1][:-4]}.pdf')
    
    plt.show()


In [ ]:
simbad_name = 'NAME_NGC_1313_X-1'
local_name =  'NGC1313'

for simbad_name, local_name in source_names_dict.items():
    xrt_img_path = f'../download_scripts/{local_name}/xrt_stack.img'
    uvot_imgs = glob(f'../download_scripts/{local_name}/*sky*img')
    for uvot_img_path in uvot_imgs:
        try:
            plot_finding_chart(simbad_name, local_name, xrt_img_path, uvot_img_path, df_closest)
        except:
            print(f'Error plotting {simbad_name} {uvot_img_path}')
            pass

In [ ]:
hdul_uv[1].data.shape

In [ ]:
cutout = Cutout2D(hdul_uv[1].data, pos, cutout_size, wcs=wcs_uv) # 50 pixel square

In [ ]:
data_no_img = plt.imread('../figures/no_img_50x50.png', format='png')

In [ ]:
source_names_dict.items()

In [ ]:
def get_simbad_sc(simbad_name):
    s = Simbad()
    source = s.query_object(simbad_name)
    sc = SkyCoord(source['RA'], source['DEC'], unit=(u.hourangle, u.deg))
    return sc

def get_nstack(hdul_uv):
        # Count number of images that were stacked
        nstack = 0
        for row in hdul_uv[0].header['HISTORY']:
            if 'img.gz' in row:
                nstack+=1
        return nstack
    
def remove_white_img(uvot_imgs):
    for img in uvot_imgs:
        if 'WHITE' in img:
            uvot_imgs.remove(img)
    return uvot_imgs

def get_uvot_img_path(uvot_imgs, uvot_filter):
    for img in uvot_imgs:
        if uvot_filter in img:
            print(uvot_filter, img)
            return img
    return False

def plot_no_img(fig, ax, iii):
    ax = plt.subplot(iii)
    ax.imshow(data_no_img)
    ax.set_xticks([])
    ax.set_yticks([])
    
    fig.add_subplot(ax)

In [ ]:
# Plot all UVOT cutout regions together
plt.rcParams.update({'font.size': 8})

fig_width = 5
fig_height = 5
cutout_size   = 50
cmap = 'hot'
textcolor = 'white'


n_imgs = 6 # One for each filter
for simbad_name, local_name in source_names_dict.items():
    fig = plt.figure(figsize=(n_imgs*fig_height,fig_height))
    
    #simbad_name, local_name = '[LM2005]_NGC_598_ULX1', 'M33'
    
    readable_name = source_names_readable[simbad_name]    
    source_distance = source_distances_mpc[simbad_name]
    sc = get_simbad_sc(simbad_name)

    uvot_imgs = glob(f'../download_scripts/{local_name}/*sky*img')
    uvot_imgs = remove_white_img(uvot_imgs)
    
    print(f'Images found {len(uvot_imgs)}')
    
    for i, uvot_filter in enumerate(['uvv','ubb','uuu','uw1','um2','uw2']):
        iii = int(f'1{n_imgs}{i+1}') # Gridspec pos
        
        uvot_img_path = get_uvot_img_path(uvot_imgs, uvot_filter)
        
        # If No image found plot empty
        if uvot_img_path == False:
            plot_no_img(fig, ax, iii)
            continue
            
        # Load UV image
        hdul_uv = fits.open(uvot_img_path)
        wcs_uv = WCS(hdul_uv[1].header)
        nstack = get_nstack(hdul_uv)
        xpos, ypos = skycoord_to_pixel(sc, wcs_uv)
        pos  = (xpos[0], ypos[0])
        try:
            cutout = Cutout2D(hdul_uv[1].data, pos, cutout_size, wcs=wcs_uv) # 50 pixel square
        except:
            plot_no_img(fig, ax, iii)
            continue
            
        xpos_cut, ypos_cut = skycoord_to_pixel(sc, cutout.wcs)
        
        # If cutout is not square plot empty
        if cutout.shape != (50,50):
            plot_no_img(fig, ax, iii)
            continue
        
        ##################
        #### Plotting ####
        ##################
        ax = plt.subplot(iii, projection=wcs_uv)
        ax.imshow(cutout.data, cmap=cmap, origin='lower', interpolation='none')

        # Plot Crosshairs
        ax.axvline(xpos_cut, lw=1.0, color=textcolor)
        ax.axhline(ypos_cut, lw=1.0, color=textcolor)

        # Plot Textbox
        bb_dict=dict(facecolor='white', edgecolor='black')
        ax.text(xpos_cut-22.5, ypos_cut+15, s=f'{readable_name}\nFILTER : {uvot_filter.upper()}\nNSTACK : {nstack}\nd = {source_distance} Mpc', color='black', size=11, bbox=bb_dict)

        # Plot Scalebar
        scalebar_size_arcsec = 5 # size in arcsec
        cdelt = cutout.wcs.wcs.cdelt[1]   # CDELT is a fits keyword that gives the size of 1 pixel in degrees
                                          # 3 arcseconds therefore corresponds to 3arcsec = cdelt * 3600
        scalebar_size_pixels  = scalebar_size_arcsec / (cdelt*3600)
        scalebar_size_parsecs = scalebar_size_arcsec*4.84814e-6 * source_distance*1e6


        ax.text(xpos_cut[0]+15, ypos_cut[0]-22, f'{scalebar_size_arcsec} arcsec\n{scalebar_size_parsecs:.2f} pc', color='white', bbox={'facecolor':'black','edgecolor':'white'})
        ax.arrow(xpos_cut[0]+15, ypos_cut[0]-17, scalebar_size_pixels, 0.0, head_width=0, head_length=0, fc='white', ec='white', width=0.50)

        # axes settings
        ax.coords[1].set_ticklabel_visible(False)
        ax.set_xlabel('RA (h:m:s)')
        ax.set_ylabel(' ')

        fig.add_subplot(ax)

    
    #fig.get_axes()[0].coords[1].set_ticklabel_visible(True)
    #fig.get_axes()[0].set_ylabel('DEC (d:m:s)')

    plt.subplots_adjust(wspace=0.01)
    #plt.tight_layout()
    plt.savefig(f'../figures/uvot_zooms/png/{readable_name}.png', bbox_inches='tight', facecolor='white')
    plt.savefig(f'../figures/uvot_zooms/pdf/{readable_name}.pdf', bbox_inches='tight', facecolor='white')
    plt.show()
    

In [ ]:
# Load UV image
simbad_name = '[LB2005]_NGC_6946_ULX3'
uvot_img_path = '../download_scripts/NGC6946/sky_img_um2.img'
readable_name = source_names_readable[simbad_name]    
source_distance = source_distances_mpc[simbad_name]
sc = get_simbad_sc(simbad_name)

hdul_uv = fits.open(uvot_img_path)
wcs_uv = WCS(hdul_uv[1].header)
nstack = get_nstack(hdul_uv)
xpos, ypos = skycoord_to_pixel(sc, wcs_uv)
pos  = (xpos[0], ypos[0])
cutout = Cutout2D(hdul_uv[1].data, pos, cutout_size, wcs=wcs_uv) # 50 pixel square


xpos_cut, ypos_cut = skycoord_to_pixel(sc, cutout.wcs)

##################
#### Plotting ####
##################
fig = plt.figure(figsize=(30,30))
ax = plt.subplot(iii, projection=wcs_uv)
ax.imshow(cutout.data, cmap=cmap, origin='lower', interpolation='none')

# Plot Crosshairs
ax.axvline(xpos_cut, lw=1.0, color=textcolor)
ax.axhline(ypos_cut, lw=1.0, color=textcolor)

# Plot Textbox
bb_dict=dict(facecolor='white', edgecolor='black')
ax.text(xpos_cut-22.5, ypos_cut+15, s=f'NGC6946 X-1\nFILTER : {uvot_filter.upper()}\nd = {source_distance} Mpc', color='black', size=11, bbox=bb_dict)

# Plot Scalebar
scalebar_size_arcsec = 5 # size in arcsec
cdelt = cutout.wcs.wcs.cdelt[1]   # CDELT is a fits keyword that gives the size of 1 pixel in degrees
                                  # 3 arcseconds therefore corresponds to 3arcsec = cdelt * 3600
scalebar_size_pixels  = scalebar_size_arcsec / (cdelt*3600)
scalebar_size_parsecs = scalebar_size_arcsec*4.84814e-6 * source_distance*1e6


ax.text(xpos_cut[0]+15, ypos_cut[0]-22, f'{scalebar_size_arcsec} arcsec\n{scalebar_size_parsecs:.2f} pc', color='white', bbox={'facecolor':'black','edgecolor':'white'})
ax.arrow(xpos_cut[0]+15, ypos_cut[0]-17, scalebar_size_pixels, 0.0, head_width=0, head_length=0, fc='white', ec='white', width=0.50)

# axes settings
ax.coords[1].set_ticklabel_visible(True)
ax.set_xlabel('RA (h:m:s)')
ax.set_ylabel('DEC (h:m:s)')
fig.add_subplot(ax)
plt.savefig('NGC6946.pdf', bbox_inches='tight')

In [ ]:
import astropy.units as u

In [ ]:
d = 4.6 * u.Mpc
arcsec = 1 * u.arcsec
x = d * np.tan(arcsec.to('rad'))
x_pc = x.to('pc')

In [ ]:
np.sin(np.pi/2)

In [ ]:
def plot_uvot_inset(simbad_name, uvot_filter, ax):
    pass

simbad_name = 'NAME_NGC_1313_X-1'
readable_name = source_names_readable[simbad_name]
local_name    = source_names_dict[simbad_name]

# Get source position from simbad
s = Simbad()
source = s.query_object(simbad_name)
sc = SkyCoord(source['RA'], source['DEC'], unit=(u.hourangle, u.deg))


# Get uvot image
uvot_imgs = glob(f'../download_scripts/{local_name}/*sky*img')
print(f'{simbad_name} {readable_name} {local_name} Found {len(uvot_imgs)} uvot images')

uvot_img_path = uvot_imgs[0]
print(f'Opening image: {uvot_img_path}')
hdul_uv = fits.open(uvot_img_path)

cutout_size   = 50

# Create UV cutout region around source
xpos, ypos = skycoord_to_pixel(sc, WCS(hdul_uv[1].header))
pos  = (xpos[0], ypos[0])
cutout = Cutout2D(hdul_uv[1].data, pos, cutout_size, wcs=WCS(hdul_uv[1].header)) # 50 pixel square

title = f'{simbad_name} | {local_name} | {readable_name} \n RA={str(sc.ra[0])} DEC={str(sc.dec[0])} \n{uvot_img_path}'

xpos_cut, ypos_cut = skycoord_to_pixel(sc, cutout.wcs)

ax = plt.subplot(projection=cutout.wcs)
ax.imshow(cutout.data)

ax.scatter(xpos_cut, ypos_cut, marker='x', color='red', label=f'{readable_name}')
ax.legend()
ax.set_xlabel(' ')
ax.set_ylabel(' ')
plt.show()



"""
# Plotting
fig = plt.figure(figsize=(10, 10))
plt.suptitle(title)
ax = plt.subplot(projection=WCS(hdul[0].header))
ax.imshow(hdul_uv[1].data, norm=LogNorm(10), cmap='hot', transform=ax.get_transform(WCS(hdul_uv[1].header)), origin='lower', interpolation='none')
ax.imshow(sources, norm=LogNorm(10), cmap='winter', interpolation='none', origin='lower')

cutout.plot_on_original(color='white', transform=ax.get_transform(WCS(hdul_uv[1].header)))
ax.axvline(xpos_xrt, lw=1.0, color='cyan')
ax.axhline(ypos_xrt, lw=1.0, color='cyan')
ax.annotate(xy=(xpos_xrt+15,ypos_xrt+5), text=readable_name, color='cyan')

# Zoom in a bit
extent = 200 # Extent in pixels
ax.set_xlim(xpos_xrt-extent, xpos_xrt+extent)
ax.set_ylim(ypos_xrt-extent, ypos_xrt+extent)

ax.set_xlabel('RA (h:m:s)')
ax.set_ylabel('DEC (d:m:s)')

# Plot the inset
left, bottom, width, height = [0.6, 0.6, 0.25, 0.25]
ax2 = fig.add_axes([left, bottom, width, height], projection=cutout.wcs)
ax2.imshow(cutout.data, origin='lower', aspect='equal', norm=LogNorm(), interpolation='none', cmap='hot')

xpos, ypos = skycoord_to_pixel(sc, cutout.wcs)
ax2.axvline(xpos, color='cyan', lw=1.0)
ax2.axhline(ypos, color='cyan', lw=1.0, label='simbad_position')
ax2.set_xlabel(' ')
ax2.set_ylabel(' ')
ax2.tick_params(axis='both', colors='white')
plt.savefig(f'../figures/finding_plots/{simbad_name}_{uvot_img_path.split("/")[-1][:-4]}.png')
plt.savefig(f'../figures/finding_plots/{simbad_name}_{uvot_img_path.split("/")[-1][:-4]}.pdf')

plt.show()

fig, ax = plt.subplots(figsize=(6,6))
"""


In [ ]:
skycoord_to_pixel

In [ ]:
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

In [ ]:
def get_stack_counts(uvot_img_paths):
    res = {}
    for img in uvot_img_paths:
        hdul_uv = fits.open(img)
        nstack = 0
        for row in hdul_uv[0].header['HISTORY']:
            if 'img.gz' in row:
                nstack+=1

        res[img] = nstack
    return res

def get_uvot_img_ext(uvot_image_path):
    uvot_img_extensions = ['ubb', 'uuu', 'uvv', 'uw1', 'uw2', 'um2']
    for f in uvot_img_extensions:
        f_str = f'_{f}.'
        if f_str in uvot_image_path:
            return f

In [ ]:
ncols = 5
nrows = 5
nimages = ncols * nrows

w = 8.0
h = w / (ncols / nrows)


fig = plt.figure(figsize=(w,h), facecolor='white')
names = list(source_distances_mpc.keys())
gs = fig.add_gridspec(nrows, ncols)
for i, g in enumerate(gs):
    simbad_name   = names[i]

    readable_name = source_names_readable[simbad_name]
    local_name    = source_names_dict[simbad_name]

    # Get source position from simbad
    s = Simbad()
    source = s.query_object(simbad_name)
    sc = SkyCoord(source['RA'], source['DEC'], unit=(u.hourangle, u.deg))

    # Get uvot image
    uvot_imgs = glob(f'../download_scripts/{local_name}/*sky*img')
    print(f'{simbad_name} {readable_name} {local_name} Found {len(uvot_imgs)} uvot images')

    
    stack_counts = get_stack_counts(uvot_imgs)
    uvot_img_path = max(stack_counts, key=stack_counts.get) # Get deepest image
    nstack = max(stack_counts.values()) # get number of deepest stack
    ext = get_uvot_img_ext(uvot_img_path)
    
    
    print(f'Opening image: {uvot_img_path}')
    hdul_uv = fits.open(uvot_img_path)

    cutout_size   = 50

    # Create UV cutout region around source
    xpos, ypos = skycoord_to_pixel(sc, WCS(hdul_uv[1].header))
    pos  = (xpos[0], ypos[0])
    try:
        cutout = Cutout2D(hdul_uv[1].data, pos, cutout_size, wcs=WCS(hdul_uv[1].header)) # 50 pixel square
    except:
        pass

    xpos_cut, ypos_cut = skycoord_to_pixel(sc, cutout.wcs)

    # Plotting
    ax = fig.add_subplot(g, projection=WCS(hdul_uv[1].header))
    ax.imshow(cutout.data, cmap='binary', interpolation='none')
    ax.axvline(xpos_cut, lw=0.5, color='black', label=f'{readable_name}\n{ext.upper()} {nstack}')
    ax.axhline(ypos_cut, lw=0.5, color='black')

    
    ax.set_xlabel(' ')
    ax.set_ylabel(' ')
    
    ax.legend(prop={'size': 6})
    #ax.coords[0].set_ticklabel(color='black', size=7)
    #ax.coords[1].set_ticklabel(color='black', size=7)
    
    
    ax.coords[0].set_ticks_visible(False)
    ax.coords[1].set_ticks_visible(False)
    ax.coords[0].set_ticklabel_visible(False)
    ax.coords[1].set_ticklabel_visible(False)
    

plt.subplots_adjust(hspace=0, wspace=0)
#plt.savefig('../figures/test_fig.pdf', bbox_inches='tight')
#plt.show()

In [ ]:
uvot_img_path = max(stack_counts, key=stack_counts.get) # Get deepest image

In [ ]:
uvot_img_path

In [ ]:
max(stack_counts, key=stack_counts.get)

In [ ]:
?cutout.wcs.printwcs

In [ ]:
0.000278888884

In [ ]:
sc1 = df_closest[df_closest['simbad_name'] == 'NAME_NGC_1313_X-1']['local_sc'].iloc[0]
sc1

In [ ]:
sc2 = df_closest[df_closest['simbad_name'] == 'NAME_NGC_1313_X-1']['simbad_sc'].iloc[0]
sc2

In [ ]:
sc_uvot_reg = df_closest[df_closest['simbad_name'] == simbad_name]['local_sc'].iloc[0]